In [1]:
import numpy as np
import pandas as pd

In [2]:
EURUSDM1 = pd.read_csv('HISTDATA2015\DAT_ASCII_EURUSD_M1_2015.csv', sep=';',
                     names=('Time','Open','High','Low','Close', ''),
                     index_col='Time', parse_dates=True)
EURUSDM1.index += pd.offsets.Hour(7) #7時間のオフセット

In [3]:
EURJPYM1 = pd.read_csv('HISTDATA2015\DAT_ASCII_EURJPY_M1_2015.csv', sep=';',
                     names=('Time','Open','High','Low','Close', ''),
                     index_col='Time', parse_dates=True)
EURJPYM1.index += pd.offsets.Hour(7) #7時間のオフセット

In [4]:
USDJPYM1 = pd.read_csv('HISTDATA2015\DAT_ASCII_USDJPY_M1_2015.csv', sep=';',
                     names=('Time','Open','High','Low','Close', ''),
                     index_col='Time', parse_dates=True)
USDJPYM1.index += pd.offsets.Hour(7) #7時間のオフセット

In [5]:
# dfのデータからtfで指定するタイムフレームの4本足データを作成する関数
def TF_ohlc(df, tf):
    x = df.resample(tf).ohlc()
    O = x['Open']['open']
    H = x['High']['high']
    L = x['Low']['low']
    C = x['Close']['close']
    ret = pd.DataFrame({'Open': O, 'High': H, 'Low': L, 'Close': C},
                       columns=['Open','High','Low','Close'])
    return ret.dropna()

In [6]:
EURUSDohlc = EURUSDM1
EURJPYohlc = EURJPYM1
USDJPYohlc = USDJPYM1

In [7]:
#描画に時間がかかるので１時間足に変換
EURUSDohlc = TF_ohlc(EURUSDM1, 'H')
EURJPYohlc = TF_ohlc(EURJPYM1, 'H')
USDJPYohlc = TF_ohlc(USDJPYM1, 'H')

In [8]:
#最初の値を１００として割合に整形する関数
def TranData(histData):
    haba=histData['Close'][0]
    histData/=haba
    return histData

In [9]:
EURUSDohlc=TranData(EURUSDohlc)
EURJPYohlc=TranData(EURJPYohlc)
USDJPYohlc=TranData(USDJPYohlc)

In [10]:
import indicators as ind #indicators.pyのインポート
EURUSDFastMA = ind.iMA(EURUSDohlc, 10) #短期移動平均
EURUSDSlowMA = ind.iMA(EURUSDohlc, 30) #長期移動平均

In [11]:
#チャートを表示
from pandas_highcharts.display import display_charts
df = pd.DataFrame({'EURUSD': EURUSDohlc['Close'],'EURJPY':EURJPYohlc['Close'],
                  'USDJPY':USDJPYohlc['Close']})
display_charts(df, chart_type="stock", title="MA cross", grid=True)

#### 移動平均線チャートを表示
from pandas_highcharts.display import display_charts
df = pd.DataFrame({'EURUSD': EURUSDohlc['Close'],'10MA':EURUSDFastMA,
                  '30MA':EURUSDSlowMA})
display_charts(df, chart_type="stock", title="MA cross", grid=True)

In [12]:
#移動平均との差分
sabun={'10sabun':EURUSDohlc['Close']-EURUSDFastMA+1,'30sabun':EURUSDohlc['Close']-EURUSDSlowMA+1}
#実体値配列とMAの差分を出したい
#キャストエラーか？

In [13]:
from pandas_highcharts.display import display_charts
df = pd.DataFrame({'EURUSD': EURUSDohlc['Close'],'１０差分': sabun['10sabun'],'30sabun':sabun['30sabun']})
display_charts(df, chart_type="stock", title="MA cross", grid=True)

In [14]:
a_list=({'t1':10,'t2':15})
a_list['t3']='4'
a_list

{'t1': 10, 't2': 15, 't3': '4'}

In [15]:
#要素番号を同時に取得
for i,dateTime in enumerate(EURUSDohlc['Close']):
    print(dateTime)
    print(i)


1.0
0
1.0
1
1.00011570057
2
0.998876051636
3
0.997272772351
4
0.997264508025
5
0.996537247318
6
0.996471132709
7
0.996082709377
8
0.995991801788
9
0.996231467248
10
0.996347167815
11
0.995636435761
12
0.996520718666
13
0.995917422853
14
0.996198409944
15
0.99487611775
16
0.994281086263
17
0.992950529743
18
0.994305879242
19
0.992843093503
20
0.992512520454
21
0.992710864283
22
0.992181947406
23
0.991991867903
24
0.991793524074
25
0.987182030049
26
0.987322523595
27
0.988636551462
28
0.987975405365
29
0.988107634585
30
0.987082858135
31
0.986843192675
32
0.986347333102
33
0.987363845226
34
0.988140691889
35
0.988016726996
36
0.987950612387
37
0.985140741475
38
0.983892828218
39
0.983471347581
40
0.984512652683
41
0.98626468984
42
0.986471297995
43
0.984917604668
44
0.985380406935
45
0.986306011471
46
0.986661377498
47
0.986479562321
48
0.986157253599
49
0.986140724947
50
0.986553941257
51
0.987008479199
52
0.986826664022
53
0.987248144659
54
0.987091122461
55
0.987487810119
56
0.9881737

In [14]:
import math
#複数コレクションを同時にループ
for (i,close),fast,slow in zip(enumerate(EURUSDohlc['Close']),EURUSDFastMA,EURUSDSlowMA):
    #nullチェック 
    #平均線の最初は空白のため
    if (not math.isnan(fast)) & (not math.isnan(slow)):
        EURUSDohlc['tankiMuki'][i]=close-fast
        EURUSDohlc['tyoukiMuki'][i]=close-slow
    else:
        EURUSDohlc['tankiMuki'][i]=0
        EURUSDohlc['tyoukiMuki'][i]=0
        
EURUSDohlc


KeyError: 'tankiMuki'

In [17]:
import myFinance as myFi
EURUSDohlc=myFi.AddHenkaritu(EURUSDohlc,5,10)

NameError: name 'tankiHenkaritu' is not defined